# Day 14: Extended Polymerization

[https://adventofcode.com/2021/day/14](https://adventofcode.com/2021/day/14)

## Description

### Part One

The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has [polymerization](https://en.wikipedia.org/wiki/Polymerization) equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains <span title="HO

HO -> OH">instructions</span> for finding the optimal polymer formula; specifically, it offers a _polymer template_ and a list of _pair insertion_ rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

    NNCB
    
    CH -> B
    HH -> N
    CB -> H
    NH -> C
    HB -> C
    HC -> B
    HN -> C
    NN -> C
    BH -> H
    NC -> B
    NB -> B
    BN -> B
    BB -> N
    BC -> B
    CC -> N
    CN -> C
    

The first line is the _polymer template_ - this is the starting point of the process.

The following section defines the _pair insertion_ rules. A rule like `AB -> C` means that when elements `A` and `B` are immediately adjacent, element `C` should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template `NNCB`, the first step simultaneously considers all three pairs:

*   The first pair (`NN`) matches the rule `NN -> C`, so element _`C`_ is inserted between the first `N` and the second `N`.
*   The second pair (`NC`) matches the rule `NC -> B`, so element _`B`_ is inserted between the `N` and the `C`.
*   The third pair (`CB`) matches the rule `CB -> H`, so element _`H`_ is inserted between the `C` and the `B`.

Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes `NCNBCHB`.

Here are the results of a few steps using the above rules:

    Template:     NNCB
    After step 1: NCNBCHB
    After step 2: NBCCNBBBCBHCB
    After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
    After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
    

This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, `B` occurs 1749 times, `C` occurs 298 times, `H` occurs 161 times, and `N` occurs 865 times; taking the quantity of the most common element (`B`, 1749) and subtracting the quantity of the least common element (`H`, 161) produces `1749 - 161 = 1588`.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. _What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?_

### Part Two

The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of _40 steps_ should do it.

In the above example, the most common element is `B` (occurring `2192039569602` times) and the least common element is `H` (occurring `3849876073` times); subtracting these produces _`2188189693529`_.

Apply _40_ steps of pair insertion to the polymer template and find the most and least common elements in the result. _What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?_


In [1]:
import numpy as np
import copy
from time import perf_counter
from collections import Counter
from functools import lru_cache

In [2]:
f = open("input.txt", "r")
rawstring = f.read()
f.close()

In [3]:
start = list(rawstring.splitlines()[0])

rules = [(list(line.split("->")[0].strip()),line.split("->")[1].strip())  for line in rawstring.splitlines()[1:] if len(line) != 0]

In [4]:
rules[0:5]

[(['K', 'S'], 'O'),
 (['S', 'P'], 'V'),
 (['O', 'H'], 'F'),
 (['V', 'C'], 'P'),
 (['B', 'O'], 'S')]

In [19]:
def getpairs(input):
    pairs = {}
    for i in range(len(input)-1):
        current = input[i]
        next = input[i+1]
        key = (current, next)
        if key in pairs.keys():
            pairs[(current, next)] += 1
        else:
            pairs[(current, next)] = 1
    return pairs

def MapPairs(pairs):
    newpairs = dict()
    for key, value in pairs.items():
        for rule in rules:
            if rule[0][0] == key[0] and rule[0][1] == key[1]:
                for newkey in [(key[0],rule[1]), (rule[1],key[1])]:
                    if newkey in newpairs.keys():
                        newpairs[newkey] += value
                    else:
                        newpairs[newkey] = value
    return newpairs

def getCount(pairs, initial):
    count = dict()
    for keys, value in pairs.items():
        for key in keys:
            if key in count.keys():
                count[key] += value
            else:
                count[key] = value
    for keys in count.keys():
        count[keys] //= 2

    for k in [initial[0], initial[-1]]:
        count[k] += 1
    return count

def getMaxDiff(count, start):
    temp = (sorted(list(getCount(count, start).values())))
    return temp[-1]-temp[0]

def solve(input, it=10):
    p = getpairs(input)
    for _ in range(it):
        p = MapPairs(p)
    return getMaxDiff(p, input)

print(solve(start, 10))
print(solve(start, 40))

2509
2827627697643


naive approach that only works until iteration 23

In [37]:
res = copy.deepcopy(start[0:3])
print(res)


@lru_cache(maxsize=None)
def applyUpdate(input: str):
    input = list(input)
    if len(input) <= 1:
        return []
    if len(input) == 2:
        for rule in rules:
            if rule[0][0] == input[0] and rule[0][1] == input[1]:
                input.insert(1, rule[1])
                break
        return input
    
    return [*applyUpdate("".join(input[:len(input)//2+1]))[:-1], *applyUpdate("".join(input[len(input)//2:]))]


def step(res):
    i = 0
    while i < (len(res)-1):
        current = res[i]
        next = res[i+1]
        for rule in rules:
            if rule[0][0] == current and rule[0][1] == next:
                res.insert(i+1, rule[1])
                i += 1
        i += 1
    return res
            

['K', 'F', 'V']
